    80/20 Para dominar os fundamentos do Tkinter e começar nossa aplicação, foque nesses conceitos principais:

Estrutura básica de uma aplicação Tkinter (janela, frames)
Sistema de grid para posicionamento
Widgets básicos (Button, Label, Frame)
Event loop e callbacks

In [1]:
import ttkbootstrap as tb
from ttkbootstrap.constants import *
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
import pandas as pd
import os


class DataManipulationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Data Manipulation System")
        self.root.geometry("900x600")

        # DataFrame principal
        self.df_original = pd.DataFrame()
        self.df = pd.DataFrame()

        # Criar widgets iniciais
        self.create_widgets()

    def create_widgets(self):
        # Botão para importar arquivo
        tb.Button(self.root, text="Importar Arquivo", command=self.import_file, bootstyle=PRIMARY).pack(pady=5)

        # Frame para exibição dos dados
        self.tree_frame = tb.Frame(self.root)
        self.tree_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)

        self.tree = tb.Treeview(self.tree_frame, columns=[], show='headings', bootstyle=INFO)
        self.tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Barras de rolagem
        self.scroll_x = tb.Scrollbar(self.tree_frame, orient=tk.HORIZONTAL, command=self.tree.xview)
        self.scroll_y = tb.Scrollbar(self.tree_frame, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(xscrollcommand=self.scroll_x.set, yscrollcommand=self.scroll_y.set)
        self.scroll_x.pack(side=tk.BOTTOM, fill=tk.X)
        self.scroll_y.pack(side=tk.RIGHT, fill=tk.Y)

        # Frame para botões
        button_frame = tb.Frame(self.root)
        button_frame.pack(pady=10)

        buttons = [
            ("Substituir Dados", self.replace_data),
            ("Tratar Nulos", self.handle_null_values),
            ("Filtrar Dados", self.filter_data),
            ("Salvar Arquivo", self.save_file),
            ("Renomear Coluna", self.rename_column),
            ("Excluir Coluna", self.delete_column),
            ("Mover Coluna", self.move_column),
            ("Excluir Linha", self.delete_row),
            ("Configurar Tipos", self.configure_column_types),
            ("Calcular Totais", self.calculate_totals),
            ("Restaurar Dados", self.restore_dataframe)
        ]

        for i, (text, command) in enumerate(buttons):
            row, col = divmod(i, 4)  # Organiza os botões em linhas
            tb.Button(button_frame, text=text, command=command, bootstyle=SUCCESS).grid(row=row, column=col, padx=5, pady=5)

    def import_file(self):
        file_path = filedialog.askopenfilename(filetypes=[("CSV/Excel", "*.csv *.xlsx *.xlsm")])
        if file_path:
            try:
                ext = os.path.splitext(file_path)[1]
                if ext == ".csv":
                    self.df_original = pd.read_csv(file_path)
                else:
                    self.df_original = pd.read_excel(file_path, engine='openpyxl')

                self.df = self.df_original.copy()
                self.display_dataframe()
            except Exception as e:
                messagebox.showerror("Erro", f"Erro ao importar arquivo: {str(e)}")

    def display_dataframe(self):
        self.tree.delete(*self.tree.get_children())

        if not self.df.empty:
            self.tree["columns"] = list(self.df.columns)
            for col in self.df.columns:
                self.tree.heading(col, text=col)
                self.tree.column(col, width=100)

            for _, row in self.df.iterrows():
                self.tree.insert('', tk.END, values=list(row))

    def replace_data(self):
        col_name = simpledialog.askstring("Coluna", "Nome da coluna para substituir dados:")
        old_value = simpledialog.askstring("Valor Antigo", "Digite o valor a ser substituído:")
        new_value = simpledialog.askstring("Novo Valor", "Digite o novo valor:")

        if col_name in self.df.columns:
            self.df[col_name] = self.df[col_name].replace(old_value, new_value)
            self.display_dataframe()
            messagebox.showinfo("Sucesso", "Dados substituídos com sucesso.")

    def handle_null_values(self):
        action = simpledialog.askstring("Ação", "Escolha entre 'remover' ou 'preencher':")
        if action == "remover":
            self.df.dropna(inplace=True)
        elif action == "preencher":
            fill_value = simpledialog.askstring("Valor", "Digite o valor para preencher nulos:")
            self.df.fillna(fill_value, inplace=True)

        self.display_dataframe()
        messagebox.showinfo("Sucesso", "Valores nulos tratados.")

    def filter_data(self):
        col_name = simpledialog.askstring("Coluna", "Nome da coluna para filtrar:")
        filter_value = simpledialog.askstring("Valor", "Digite o valor para filtrar:")

        if col_name in self.df.columns:
            self.df = self.df[self.df[col_name] == filter_value]
            self.display_dataframe()
            messagebox.showinfo("Sucesso", "Dados filtrados.")

    def rename_column(self):
        col_name = simpledialog.askstring("Renomear Coluna", "Nome da coluna a ser renomeada:")
        new_name = simpledialog.askstring("Novo Nome", "Digite o novo nome da coluna:")

        if col_name in self.df.columns:
            self.df.rename(columns={col_name: new_name}, inplace=True)
            self.display_dataframe()
            messagebox.showinfo("Sucesso", "Coluna renomeada.")

    def delete_column(self):
        col_name = simpledialog.askstring("Excluir Coluna", "Nome da coluna a ser excluída:")
        if col_name in self.df.columns:
            self.df.drop(columns=[col_name], inplace=True)
            self.display_dataframe()
            messagebox.showinfo("Sucesso", "Coluna excluída.")

    def move_column(self):
        col_name = simpledialog.askstring("Mover Coluna", "Nome da coluna a ser movida:")
        new_position = simpledialog.askinteger("Nova Posição", "Digite a nova posição:")

        if col_name in self.df.columns and new_position is not None:
            cols = list(self.df.columns)
            cols.remove(col_name)
            cols.insert(new_position, col_name)
            self.df = self.df[cols]
            self.display_dataframe()
            messagebox.showinfo("Sucesso", "Coluna movida.")

    def delete_row(self):
        row_index = simpledialog.askinteger("Excluir Linha", "Índice da linha:")
        if 0 <= row_index < len(self.df):
            self.df.drop(index=self.df.index[row_index], inplace=True)
            self.display_dataframe()
            messagebox.showinfo("Sucesso", "Linha excluída.")

    def configure_column_types(self):
        col_name = simpledialog.askstring("Configurar Tipos", "Nome da coluna:")
        col_type = simpledialog.askstring("Tipo", "int, float, str, date:")

        if col_name in self.df.columns:
            try:
                type_map = {"int": int, "float": float, "str": str, "date": lambda x: pd.to_datetime(x, errors='coerce')}
                self.df[col_name] = self.df[col_name].apply(type_map[col_type])
                self.display_dataframe()
                messagebox.showinfo("Sucesso", "Tipo configurado.")
            except:
                messagebox.showerror("Erro", "Erro ao configurar tipo.")

    def calculate_totals(self):
        col_name = simpledialog.askstring("Cálculo", "Nome da coluna:")
        if col_name in self.df.columns:
            total = self.df[col_name].sum()
            messagebox.showinfo("Total", f"Soma: {total}")

    def restore_dataframe(self):
        self.df = self.df_original.copy()
        self.display_dataframe()
        messagebox.showinfo("Sucesso", "Data restaurada.")

    def save_file(self):
        file_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel", "*.xlsx")])
        if file_path:
            self.df.to_excel(file_path, index=False, engine='openpyxl')
            messagebox.showinfo("Sucesso", "Arquivo salvo.")


if __name__ == "__main__":
    root = tb.Window(themename="darkly")
    app = DataManipulationApp(root)
    root.mainloop()
